In [68]:
#параметры базы проекта
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432


In [3]:
#импортируем библиотеки
import psycopg2
import pandas as pd

In [69]:
#Создаём соединение с заданными параметрами:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

# Предварительный анализ данных

In [5]:
#считаем количество вакансий в базе
vac_count=f'''select count(distinct id)
           FROM vacancies
        '''
        
vac_count_table = pd.read_sql_query(vac_count, connection)
vac_count_table        

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\508396300.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vac_count_table = pd.read_sql_query(vac_count, connection)


,count
0,49197


In [6]:
#считаем количество работодателей
query_employers = f'''select count(distinct id)
    FROM employers
        '''
        
employers = pd.read_sql_query(query_employers, connection)
employers   

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\171248567.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers = pd.read_sql_query(query_employers, connection)


,count
0,23501


In [7]:
#считаем количество регионов
query_areas = f'''select count(distinct id)
    FROM areas
    '''
    
areas_count = pd.read_sql_query(query_areas, connection)   
areas_count 

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\4030798710.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_count = pd.read_sql_query(query_areas, connection)


,count
0,1362


In [8]:
#считаем сферы деятельности
query_areas = f'''select count(distinct id)
    FROM industries
    '''
    
ind_count = pd.read_sql_query(query_areas, connection)   
ind_count 

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\3497120562.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ind_count = pd.read_sql_query(query_areas, connection)


,count
0,294


In [114]:
#Посмотрим на названия компаний с наибольшим количеством опубликованных вакансий, для этого сгруппируем таблицы вакансий и работодателей
modul_1 = f'''select v.employer_id, e.name, count(v.id)
    from vacancies v
    JOIN employers e on v.employer_id = e.id
    group by 1,2
    order by 3 desc
    LIMIT 5
    '''
    
module_1 = pd.read_sql_query(modul_1, connection)
module_1    

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\2751357370.py:10: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employer_id,name,count
0,1740,Яндекс,1933
1,2748,Ростелеком,491
2,78638,Тинькофф,444
3,3529,СБЕР,428
4,39305,Газпром нефть,331


Из предварительного анализа данных мы можем сделать вывод, что в среднем на 1 работодателя приходится более одной вакансии. 
При этом наибольшее количечество вакансий приходится на Яндекс, Ростелеком, Тинькофф и Сбер, что может быть обусловленно быстрым ростом данных фирм и
расширеним сфер деятельности.


# Детальный анализ вакансий

In [115]:
#Сколько вакансий в каждом регионе.
#Объединяем таблицы с вакансиями с таблицей по регионам по ключу id региона 
query_4_1 = f'''select a.name,
        COUNT (v.id)
           FROM vacancies v
           JOIN areas a on a.id = v.area_id
           GROUP by 1
           ORDER by 2 DESC  
           limit 7
        '''

top = pd.read_sql_query(query_4_1, connection)
top

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\3413114733.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670


In [83]:
#У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?
query_4_2 = f'''select count(id)
           FROM vacancies 
           WHERE (salary_from IS NOT NULL) or (salary_to IS NOT NULL)
        '''

df = pd.read_sql_query(query_4_2, connection)
df        

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\3689439606.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


,count
0,24073


In [116]:
#Найдите средние значения для нижней и верхней границы зарплатной вилки. 
query_4_3 = f'''select ROUND(AVG(salary_from),0) salary_from_avg,
            ROUND(AVG(salary_to),0) salary_to_avg
           FROM vacancies 
        '''
avg = pd.read_sql_query(query_4_3, connection)
avg        

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\3000292867.py:6: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,salary_from_avg,salary_to_avg
0,71065.0,110537.0


In [87]:
#Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule)
# и типа трудоустройства (employment), используемого в вакансиях. 
query_4_4 = f'''select schedule,
      employment
FROM vacancies 
GROUP by 1, 2
ORDER by COUNT (*) DESC
Limit 2
        '''
        
schedule = pd.read_sql_query(query_4_4, connection)
schedule        

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\3500474554.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schedule = pd.read_sql_query(query_4_4, connection)


,schedule,employment
0,Полный день,Полная занятость
1,Удаленная работа,Полная занятость


In [86]:
#Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) 
# в порядке возрастания количества вакансий, в которых указан данный вариант опыта.
query_4_5 = f'''select experience
FROM vacancies 
GROUP by 1
ORDER by COUNT (*) 
        '''
experience = pd.read_sql_query(query_4_5, connection)
experience        

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\2039939562.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  experience = pd.read_sql_query(query_4_5, connection)


,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


Как мы видим, стандартный график работы "Полный день - полная занятость" все еще является наиболее популярным у работодателей.
Также неизменным остается то, что основным опытом работы ставят 1-3 года, что обусловлено тенденций нанимать линейный персонал с продвижением отличившихся сотрудников по карьерной лестнице внутри компании.
Лидерами по количеству вакансий неизменно остаются Москва и СПб.

# Анализ работодателей

In [88]:
#какие работодатели находятся на первом и пятом месте по количеству вакансий.
query_5_1 = f'''select e.name
FROM employers e 
JOIN vacancies v on e.id = v.employer_id
GROUP by 1
ORDER by COUNT (*) DESC
LIMIT 5'''

comp_count = pd.read_sql_query(query_5_1, connection)
comp_count

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\327040166.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  comp_count = pd.read_sql_query(query_5_1, connection)


,name
0,Яндекс
1,Ростелеком
2,Тинькофф
3,СБЕР
4,Газпром нефть


In [110]:
#5.2 Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.
# Впишите его название в поле ниже в том виде, который вернул запрос.
query_5_2 = f'''select  a.name, count(e.id) employer_count, count(v.id) vac_count
from areas a
left join employers e on a.id = e.area
left join vacancies v on a.id = v.area_id
group by a.name
having count(v.id) = 0
order by 2 desc
LIMIT 5''' 

areas_5_2 = pd.read_sql_query(query_5_2, connection)
areas_5_2

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\456593431.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,employer_count,vac_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


In [23]:
#Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии
query_5_3 = f'''select v.employer_id, count(distinct v.area_id)
from vacancies v 
GROUP by 1 
ORDER BY 2 DESC
LIMIT 5 
''' 

areas_5_3 = pd.read_sql_query(query_5_3, connection)
areas_5_3

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\1943299681.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_5_3 = pd.read_sql_query(query_5_3, connection)


,employer_id,count
0,1740,181
1,2748,152
2,5724811,116
3,5130287,88
4,3682876,71


In [25]:
#Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.
query_5_4 = f'''select count(e.id)
from employers e 
LEFT JOIN employers_industries i on e.id = i.employer_id 
WHERE i.industry_id IS NULL
'''

null_ind = pd.read_sql_query(query_5_4, connection)
null_ind

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\4111147125.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  null_ind = pd.read_sql_query(query_5_4, connection)


,count
0,8419


In [26]:
#Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний,
#у которых указано четыре сферы деятельности.
query_5_5 = f'''select e.name
from employers e 
JOIN  employers_industries i on e.id = i.employer_id
GROUP by 1
HAVING count(industry_id) = 4
Order by 1
LIMIT 3
'''

ind_5_5 = pd.read_sql_query(query_5_5, connection)
ind_5_5

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\3733888767.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ind_5_5 = pd.read_sql_query(query_5_5, connection)


,name
0,101 Интернет
1,21vek.by
2,2ГИС


In [27]:
#у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения»
query_5_6 = f'''select count(e.employer_id)
from employers_industries e 
JOIN  industries i on i.id = e.industry_id
WHERE i.name = 'Разработка программного обеспечения'
'''

dev_count = pd.read_sql_query(query_5_6, connection)
dev_count

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\3168613143.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dev_count = pd.read_sql_query(query_5_6, connection)


,count
0,3553


In [118]:
#5.7 Для компании «Яндекс» выведите список регионов-миллионников , в которых представлены вакансии компании,
# вместе с количеством вакансий в этих регионах. Также
# добавьте строку Total с общим количеством вакансий компании, собранных в этой таблице:
cities = ('Москва', 'Санкт-Петербург', 'Новосибирск', 
          'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 
          'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


query_5_7 =f'''
select 
    a.name,
    count(v.id)
from vacancies v
left join employers e on v.employer_id = e.id
left join areas a on a.id = v.area_id
where e.name = 'Яндекс'
and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 
          'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 
          'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')
group by a.name
order by 2 desc
'''

city_mln = pd.read_sql_query(query_5_7,connection)
city_mln

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\643698801.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Москва,54
1,Санкт-Петербург,42
2,Екатеринбург,39
3,Нижний Новгород,36
4,Новосибирск,35
5,Воронеж,32
6,Краснодар,30
7,Самара,26
8,Уфа,26
9,Казань,25


In [98]:
module2 = f'''select v.employer_id, e.name, count(distinct v.area_id) as area_count
from vacancies v 
JOIN employers e on e.id = v.employer_id
GROUP by 1,2
ORDER BY 3 DESC
LIMIT 30
''' 

module_2 = pd.read_sql_query(module2, connection)

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\1635039462.py:9: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [92]:
import plotly.express as px

In [100]:
fig = px.histogram(module_2, x = 'name', y = 'area_count', title = 'ТОП 30 работодателей по количеству регионов')
fig.show()

по данному блоку мы видим, что лидерами по количеству регионов остаются Яндекс и Ростелеком, что соответствует и большому количеству Вакансий.
Остальные ранее отмеченные компании-лидеры по количеству вакансий не так распространены по регионам.

Также можно отметить, что некоторые компании в качестве региона отмечают страну (Россия, Беларусия и т.д.), при этом вакансии там не размещены,что может говорить о возможности удаленной работы.

# Предметный анализ

In [101]:
#Сколько вакансий имеет отношение к данным
query_6_1 = f'''select 
    count(v.id)
from vacancies v
WHERE v.name LIKE '%DATA%' or v.name like '%ДАНН%' 
or v.name like '%Data%' or v.name like '%Данн%' OR v.name like '%data%' or v.name like '%данн%'
'''

data = pd.read_sql_query(query_6_1, connection)
data

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\638699296.py:9: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1771


In [108]:
#процент от общего количества вакансий
data/vac_count_table*100

,count
0,3.599813


In [102]:
#Сколько есть подходящих вакансий для начинающего дата-сайентиста?
query_6_2 = f'''select 
    count(*)
from vacancies v
where (v.name ilike '%data scientist%'
                or v.name ilike '%data science%'
                or v.name ilike '%исследователь данных%'
                or (v.name ilike '%ML%' 
                and v.name not ilike '%HTML%')
                or v.name ilike '%machine learning%'
                or v.name ilike '%машинн%обучен%')
                and (v.name ilike '%junior%' or v.experience ilike 'нет опыта' or v.employment ilike 'стажировка')
                '''
                
vacs = pd.read_sql_query(query_6_2, connection)
vacs                

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\2000565130.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,51


In [103]:
#Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?
query_6_3 = f'''select 
    count(*)
from vacancies v
where (v.name ilike '%data scientist%'
                or v.name ilike '%data science%'
                or v.name ilike '%исследователь данных%'
                or (v.name like '%ML%' 
                and v.name not ilike '%HTML%')
                or v.name ilike '%machine learning%'
                or v.name ilike '%машинн%обучен%')
                and (v.key_skills ilike '%SQL%' or v.key_skills ilike '%postgres%')
                ''' 
                
vacs_2 = pd.read_sql_query(query_6_3, connection)
vacs_2                                

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\50010741.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,201


In [104]:
#Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.
query_6_4 = f'''select 
    count(*)
from vacancies v
where (v.name ilike '%data scientist%'
                or v.name ilike '%data science%'
                or v.name ilike '%исследователь данных%'
                or (v.name like '%ML%' 
                and v.name not ilike '%HTML%')
                or v.name ilike '%machine learning%'
                or v.name ilike '%машинн%обучен%')
                and (v.key_skills ilike '%python%')
                '''
                
vac_py = pd.read_sql_query(query_6_4, connection)
vac_py                

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\1029699103.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,351


In [55]:
#Сколько ключевых навыков в среднем указывают в вакансиях для DS?
query_6_5 = f'''select 
    round(avg(length(key_skills)
             -length(replace(key_skills, CHR(9), ''))+1), 2)
from vacancies v
where (v.key_skills is not null) and 
(v.name ilike '%data scientist%'
                or v.name ilike '%data science%'
                or v.name ilike '%исследователь данных%'
                or (v.name like '%ML%' 
                and v.name not ilike '%HTML%')
                or v.name ilike '%machine learning%'
                or v.name ilike '%машинн%обучен%')
                '''
                
skills = pd.read_sql_query(query_6_5, connection)
skills
               

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\2737810367.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  skills = pd.read_sql_query(query_6_5, connection)


,round
0,6.41


In [105]:
#Какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience)
query_6_6 = f'''SELECT 
    v.experience, 
    ROUND(AVG(coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)), 0) 
FROM vacancies v 
WHERE (v.name ilike '%data scientist%'
                or v.name ilike '%data science%'
                or v.name ilike '%исследователь данных%'
                or (v.name like '%ML%' 
                and v.name not ilike '%HTML%')
                or v.name ilike '%machine learning%'
                or v.name ilike '%машинн%обучен%')
GROUP by 1                
''' 

final = pd.read_sql_query(query_6_6, connection)
final

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\3210971359.py:16: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,round
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


По данному блоку анализа мы видим, что несмотря на популярность направления DS, вакансии составляют 3,5% от всех вакансий. Для начинающего специалиста количество всего 51 вакансия.
Также мы видим прямую зависимость роста з/п относительно стажа потенциального сотрудника. Неуказанная з/п для опыта "Более 6 лет" говорит о том, что в данном случае з/п устанавливается по результатам собеседования. 

In [112]:
fig_fin = px.histogram(top, x = 'name', y = 'count', title = 'Топ городов по количеству вакансий')
fig_fin.show()

In [113]:
indust = f'''select ii.name as industry, count(e.id) 
from employers_industries i
left join employers e on e.id = i.employer_id
left join industries ii on i.industry_id = ii.id
group by 1 
order by 2 Desc
LIMIT 10'''

top_ind = pd.read_sql_query(indust, connection)
top_ind

C:\Users\tanya\AppData\Local\Temp\ipykernel_20172\2269662170.py:9: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,industry,count
0,Разработка программного обеспечения,3553
1,"Системная интеграция, автоматизации технологи...",2993
2,"Интернет-компания (поисковики, платежные систе...",1675
3,"Маркетинговые, рекламные, BTL, дизайнерские, E...",798
4,Консалтинговые услуги,662
5,Кадровые агентства,389
6,"Электронно-вычислительная, оптическая, контрол...",374
7,Розничная сеть (продуктовая),368
8,Интернет-магазин,351
9,"Архитектура, проектирование",346


# Общие выводы

* Согласно предоставленным данным мы видим, что основное количество вакансий опубликовано для Москвы и Санкт-Петербурга и других курпных городов России и СНГ, что указывает на сохранение тенденции привлечения людей в центр регионов.

* Несмотря на то, что вакансий непосредственно по направлению DS всего 3,5%, можно заметить, что на разработку ПО приходится 3553 вакансии, а нас ситемную интеграцию - 2993. По этим данным можно сказать, что сфера IT все еще одна из самых распространенных на рынке труда и молодым специалистам стоит обратить на это внимание.

* Как указано ранее, специалистам DS без опыта нужно быть готовым к не самой высокой з/п, однако по мере его накопления з/п будет расти значительно.

* Видна тенденция компаний нанимать людей с опытом работы от 1 года, что может быть связано с нежелание найма топ-менеджеров - компании в данном случае отдают предпочтение повышению сотрудников с наймом на его предыдущее место.

* Несмотря на неизменное лидирующую позицию графика "Полный день - полная занятость" на втором месте стоит удаленная работа с полной занятостью, что говорит о популязации данного режима работы.

* Яндекс является абсолютным лидером по количеству вакансий и регионов.